# Modeling Crop Yield Anomaly
## Python modules

In [1]:
## import warnings
import time
import math
import os
import glob
from pyhere import here
from datetime import date
import re

import numpy as np
import pandas as pd
import geopandas

import pyarrow
import itertools
import multiprocessing
import p_tqdm

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, LeaveOneGroupOut, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from scipy.stats import spearmanr,  pearsonr

In [2]:
def str2bool(string):
    return string.lower() in ("yes", "true", "t", "1")

point_pattern = re.compile("20k-points")
wa_pattern = re.compile("cm-False")

In [3]:
data_dir = here("data")
directory = here("data", "random_features", "summary")
today = date.today().strftime("%Y-%m-%d")
files = os.listdir(directory)
files = [f for f in files if f not in ('.gitkeep', '.ipynb_checkpoints')]
files = [f for f in files if not (bool(point_pattern.search(f)) & bool(wa_pattern.search(f)))]
len(files)

44

In [4]:
crop_df = pd.read_csv(here('data', 'crop_yield', 'cfs_maize_districts_zambia_2009_2022.csv'))
crop_df = crop_df.set_index(['district', 'year'])[['log_yield_mt_anomaly']]

In [5]:
def model(params):
#########################################     SET PARAMS    #########################################
    file         = params
    f            = file.split(sep="_")
    satellite    = f[0]
    bands        = f[1].replace("bands-", "")
    country_code = f[2]
    points       = f[3].replace("k-points", "")
    num_features = f[4].replace("-features", "")
    yrs          = f[5].replace("yr-", "").split(sep="-")
    mns          = f[6].replace("mn-", "").split(sep="-")
    limit_months = str2bool(f[7].replace("lm-", ""))
    crop_mask    = str2bool(f[8].replace("cm-", ""))
    weighted_avg = str2bool(f[9].replace("wa-", ""))
    years        = range(int(yrs[0]), int(yrs[1])+1)
    month_range  = list(range(int(mns[0]), int(mns[1])+1))
    
    alphas = {'alpha': np.logspace(-8, 8, base = 10, num = 17)}
    kfold = KFold()
    logo = LeaveOneGroupOut()
    ridge = Ridge() 
#########################################     READ DATA    #########################################
    fn = f"{directory}/{file}"
    features = pd.read_feather(fn)
     
    drop_cols = ['district', 'year', 'yield_mt', 'crop_perc']
            
    if weighted_avg:
        drop_cols.remove("crop_perc")
    else:
        pass
    
    features = features.set_index(drop_cols)

#########################################     CALCULATE ANOMALY   #########################################
    features['yield_mt'] = np.log10(features['yield_mt'] + 1)
    features.set_index(['year', 'district'], inplace=True)
    var_cols = features.columns
    features = features[var_cols] - features.groupby(['district'], as_index=True)[var_cols].transform('mean')
    features.reset_index(drop=False, inplace=True)

#########################################     K-FOLD SPLIT    #########################################
    x_all = features.drop(drop_cols, axis = 1) 
    y_all = features.yield_mt
    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=0)
    
#########################################     K-FOLD  CV   ###########################################
    ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
    kfold_ridge_reg = GridSearchCV(ridge, alphas, scoring = 'r2', cv = kfold)
    kfold_ridge_reg.fit(x_train, y_train)
    kfold_best_model = kfold_ridge_reg.best_estimator_
    ### PREDICT - PREDICTING WITH BEST HYPERPARAMETER
    kfold_val_predictions = cross_val_predict(kfold_best_model, X = x_train, y = y_train, cv = kfold)   
    y_pred_train_k = kfold_best_model.predict(x_train)
    y_pred_test_k  = kfold_best_model.predict(x_test)

#########################################     LOGO SPLIT   ###########################################
    x_train_g = features[features.year < max(features.year)].drop(drop_cols, axis=1)
    y_train_g = features[features.year < max(features.year)].yield_mt
    g_train_g = features[features.year < max(features.year)].year.ravel()

    x_test_g = features[features.year == max(features.year)].drop(drop_cols, axis=1)
    y_test_g = features[features.year == max(features.year)].yield_mt
    g_test_g = features[features.year == max(features.year)].year

#########################################     LOGO  CV   ###########################################
    ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
    logo_ridge_reg = GridSearchCV(ridge, alphas, scoring='r2', cv=logo)
    logo_ridge_reg.fit(x_train_g, y_train_g, groups=g_train_g)
    logo_best_model = logo_ridge_reg.best_estimator_
    ### PREDICT - PREDICTING WITH BEST HYPERPARAMETER
    logo_val_predictions = cross_val_predict(logo_best_model, X=x_train_g, y=y_train_g, groups=g_train_g, cv=logo)   
    logo_train_pred = logo_best_model.predict(x_train_g)
    logo_test_pred  = logo_best_model.predict(x_test_g)
    
#########################################     SAVE RESULTS    #########################################
    d = {
        'country': country_code,
        'satellite': satellite,
        'bands': bands,
        'num_features': num_features,
        'points': points, 
        'month_range': f'{min(month_range)}-{max(month_range)}',
        
        'limit_months': limit_months,
        'crop_mask': crop_mask,
        'weighted_avg': weighted_avg,
        
        'kfold_total_n': len(x_all),
        'kfold_train_n': len(x_train),
        'kfold_test_n': len(x_test),
        
        'kfold_best_reg_param': list(kfold_ridge_reg.best_params_.values())[0],
        'kfold_mean_of_val_R2s': kfold_ridge_reg.best_score_,
        'kfold_val_R2': r2_score(y_train, kfold_val_predictions),
        'kfold_val_r' : pearsonr(kfold_val_predictions, y_train)[0],
        'kfold_val_r2' : pearsonr(kfold_val_predictions, y_train)[0] ** 2,
        
        'kfold_train_R2': r2_score(y_train, y_pred_train_k),
        'kfold_train_r': pearsonr(y_pred_train_k, y_train)[0],
        'kfold_train_r2': pearsonr(y_pred_train_k, y_train)[0] ** 2,
        
        'kfold_test_R2': r2_score(y_test, y_pred_test_k),
        'kfold_test_r': pearsonr(y_pred_test_k, y_test)[0],
        'kfold_test_r2': pearsonr(y_pred_test_k, y_test)[0] ** 2,
        
        'logo_total_n': len(x_all),
        'logo_train_n': len(x_train),
        'logo_test_n': len(x_test),    
        
        'logo_best_reg_param': list(logo_ridge_reg.best_params_.values())[0],      
        'logo_mean_of_val_R2s' : logo_ridge_reg.best_score_,
        'logo_val_R2' : r2_score(y_train_g, logo_val_predictions),
        'logo_val_r' : pearsonr(logo_val_predictions, y_train_g)[0],
        'logo_val_r2' : pearsonr(logo_val_predictions, y_train_g)[0] ** 2,
        
        'logo_train_R2': r2_score(y_train_g, logo_train_pred),
        'logo_train_r': pearsonr(logo_train_pred, y_train_g)[0],
        'logo_train_r2': pearsonr(logo_train_pred, y_train_g)[0] ** 2,
        
        'logo_test_R2': r2_score(y_test_g, logo_test_pred),
        'logo_test_r': pearsonr(logo_test_pred, y_test_g)[0],
        'logo_test_r2': pearsonr(logo_test_pred, y_test_g)[0] ** 2,
    }
    return pd.DataFrame(data=d, index=[0])

In [6]:
%%time     
##### With progress bar
workers = 44 ## os.cpu_count()
if __name__ == "__main__":
    output = []
    for result in p_tqdm.p_map(model, files, num_cpus=workers):
        output.append(result)
    results = pd.concat(output).reset_index(drop=True)
    today = date.today().strftime("%Y-%m-%d")
    file_name = f'anomaly_results_{today}.csv'
    print(f"Saving results as: {file_name}\n\n")           
    results.to_csv(here("data","results", file_name), INDEX = False)

  0%|          | 0/44 [00:00<?, ?it/s]

Saving results as: anomaly_results_2022-11-16.csv


CPU times: user 603 ms, sys: 486 ms, total: 1.09 s
Wall time: 17min 6s


In [7]:
results

,country,satellite,bands,num_features,points,month_range,limit_months,crop_mask,weighted_avg,kfold_total_n,...,logo_mean_of_val_R2s,logo_val_R2,logo_val_r,logo_val_r2,logo_train_R2,logo_train_r,logo_train_r2,logo_test_R2,logo_test_r,logo_test_r2
0,ZMB,sentinel-2-l2a,2-3-4,1000,4,4-9,True,True,True,506,...,-0.416405,-0.106623,-0.395369,0.156317,0.053905,0.383456,0.147038,-0.050376,-0.022882,0.000524
1,ZMB,sentinel-2-l2a,2-3-4-8,1000,15,1-12,False,False,False,499,...,-0.443454,-0.129104,-0.541464,0.293184,0.000088,0.131525,0.017299,-0.045141,0.083630,0.006994
2,ZMB,sentinel-2-l2a,2-3-4,1000,4,4-9,True,False,True,506,...,-0.415841,-0.106298,-0.393085,0.154516,0.054329,0.383703,0.147228,-0.050964,-0.031325,0.000981
3,ZMB,landsat-8-c2-l2,1-2-3-4-5-6-7,1000,15,4-9,True,False,True,648,...,-0.499165,-0.086000,-0.529903,0.280797,0.000041,0.195374,0.038171,-0.641621,0.033419,0.001117
4,ZMB,sentinel-2-l2a,2-3-4-8,1000,15,4-9,True,True,True,506,...,-0.440767,-0.129005,-0.541357,0.293068,0.000018,0.103877,0.010790,-0.042614,0.024994,0.000625
5,ZMB,sentinel-2-l2a,2-3-4-8,1000,15,1-12,False,True,True,492,...,-0.446893,-0.130723,-0.544862,0.296874,0.000081,0.129518,0.016775,-0.047611,0.156865,0.024607
6,ZMB,landsat-8-c2-l2,1-2-3-4-5-6-7,1000,15,4-9,True,False,False,648,...,-0.499159,-0.085998,-0.529878,0.280771,0.000029,0.210990,0.044517,-0.641593,-0.090069,0.008112
7,ZMB,sentinel-2-l2a,2-3-4,1000,4,4-9,True,False,False,506,...,-0.419017,-0.107983,-0.416805,0.173726,0.049845,0.410693,0.168668,-0.053029,-0.090125,0.008123
8,ZMB,sentinel-2-l2a,2-3-4,1000,15,4-9,True,False,False,506,...,-0.440985,-0.128914,-0.541227,0.292927,0.001933,0.398193,0.158557,-0.042431,-0.217554,0.047330
9,ZMB,sentinel-2-l2a,2-3-4,1000,15,1-12,False,False,True,499,...,-0.443461,-0.128582,-0.541333,0.293042,0.003802,0.519806,0.270198,-0.044696,-0.324898,0.105558
